In [1]:
import numpy as np
from astropy.io import fits
from astropy.table import Table, Column, join
from astropy.wcs import WCS

In [2]:
def fitsextract(filename, stride=[1,1,1], keepref=True, keepnan=True, header=None, 
                lbl='index',bunit=None, col_name=None, zselect=None, suffix=''):
    hdu   = fits.open(filename)[0]
    data  = hdu.data
    hdr   = hdu.header if header is None else header
    bunit = hdr['BUNIT'] if bunit is None else bunit
    w     = WCS(hdr)
    print('RA ref is',w.wcs.crval[0])
    print('DEC ref is',w.wcs.crval[1])
    ndim  = len(data.shape)
    iscube = (ndim > 2 and data.shape[ndim-3] > 1)
    pseudo = (hdr['ctype3'] == '')
    if iscube and not pseudo:
        print('This is a data cube of shape', data.shape)
        data = np.squeeze(data)
        if len(data.shape) > 3:
            raise ('Data cannot be squeezed to three dimensions')
        naxis = 3
        nz,ny,nx = data.shape
        ix,iy,iz = np.meshgrid(np.arange(nx), np.arange(ny), np.arange(nz),
                               indexing='ij')
        tab = Table([np.ravel(ix), np.ravel(iy), np.ravel(iz)],
                  names=('ix','iy','iz'),dtype=('i4','i4','i4'))
        # Get the pixel coordinates as tuples
        wcsin = (np.array([tab['ix'],tab['iy'],tab['iz']])).T
    else:
        print('This is an image of shape', data.shape)
        data = np.squeeze(data)
        naxis = 2
        if pseudo:
            nz,ny,nx = data.shape
        else:
            ny,nx = data.shape
        ix,iy = np.meshgrid(np.arange(nx), np.arange(ny), indexing='ij')
        tab = Table([np.ravel(ix), np.ravel(iy)],
                    names=('ix','iy'),dtype=('i4','i4'))
        # Get the pixel coordinates as tuples
        wcsin = (np.array([tab['ix'],tab['iy']])).T
    wfix = w.sub(naxis)
    wcsout = wfix.wcs_pix2world(wcsin,0)
    col_ra = Column(wcsout.T[0]-w.wcs.crval[0], name='ra_off',  dtype='f4', unit='deg')
    col_dc = Column(wcsout.T[1]-w.wcs.crval[1], name='dec_off', dtype='f4', unit='deg')
    tab.add_columns([col_ra,col_dc])
    if iscube and not pseudo:
        col_vel = Column(wcsout.T[2]/1000., name='vel', dtype='f4', unit='km/s')
        tab.add_column(col_vel)
    # Use order = 'F' because the velocity axis is first
    if pseudo:
        zsel = range(nz) if zselect is None else zselect
        for i in zsel:
            col_data = Column(np.ravel(data[i],order='F'), 
                              name=hdr[lbl+str(i)]+suffix, dtype='f4', unit=bunit)
            tab.add_column(col_data)
    else:
        cname = 'imgdata' if col_name is None else col_name
        col_data = Column(np.ravel(data,order='F'), name=cname, dtype='f4', unit=bunit)
        tab.add_column(col_data)
    # Select the desired rows from the full table
    idx = ['ix', 'iy', 'iz']
    rem = [0, 0, 0]
    select = [[],[],[]]
    if keepref:
        for i in range(naxis):
            crpix = wfix.wcs.crpix[i]
            if crpix < 1 or crpix > hdr['naxis'+str(i+1)] or not crpix.is_integer():
                print('Cannot use keepref on axis {}: crpix={}'.format(i+1,crpix))
                continue
            else:
                print('Axis {}: crpix={}'.format(i+1,crpix))
                rem[i] = int(crpix-1) % stride[i]
        print('Remainder: ',rem)
    for i in range(naxis):
        select[i]=np.where(tab[idx[i]] % stride[i] == rem[i])[0]
    xy = np.intersect1d(select[0], select[1])
    if iscube and not pseudo:
        xyz = np.intersect1d(xy, select[2])
        if len(xyz) < len(tab):
            newtab = tab[xyz]
            tab = newtab
    else:
        if len(xy) < len(tab):
            newtab = tab[xy]
            tab = newtab
    # Remove NaN rows if desired
    if not keepnan:
        newtab = tab[~np.isnan(tab['imgdata'])]
        tab = newtab
    return tab



In [3]:
# Still need to regrid to match EDGE CO data.  Should treat 0's as NaN.

In [4]:
# First read the native resolution file since it has the astrometry
file='fitsdata/xindices.CS.NGC4047.cube.fits.gz'
hdr = fits.getheader(file)
tab0 = fitsextract(file, keepnan=True, stride=[2,2,1], bunit='Angstrom', lbl='index', suffix='_rg')

RA ref is 180.7109389
DEC ref is 48.63599793
This is an image of shape (18, 72, 77)
Axis 1: crpix=38.0
Axis 2: crpix=33.0
Remainder:  [1, 0, 0]


In [5]:
# Then read the smoothed file
file='fitsdata/indices.CS.NGC4047.cube.fits.gz'
tab1 = fitsextract(file, keepnan=True, stride=[2,2,1], header=hdr, bunit='Angstrom', lbl='index', suffix='_sm')

RA ref is 180.7109389
DEC ref is 48.63599793
This is an image of shape (18, 72, 77)
Axis 1: crpix=38.0
Axis 2: crpix=33.0
Remainder:  [1, 0, 0]


In [7]:
joint=join(tab0,tab1)
#joint.show_in_notebook()